# 2. Data Acquisition
<strong>Quejas de los Usuarios de Servicios Financieros en Estados Unidos</strong>

---


Primero, se importan las librerías requeridas:

In [1]:
# Importación de datos y gestión de rutas
import sys
import os
sys.path.append('..') # Folder principal del projecto
from config import RAW_DATA_DIR, RAW_DATA_PATH

import gdown

# Manejo de datos
import pandas as pd

#Filtrar warnings
import warnings
warnings.filterwarnings('ignore')

## 2.1 Cargar el dataset

<p style="text-align: justify;">El dataset "US Consumer Financial Complaints", obtenido de la plataforma Kaggle, presenta de manera detallada un listado de quejas por parte de diversos usuarios de servicios financieros en Estados Unidos hacia la Oficina para la Protección Financiera del Consumidor. A continuación, se realizará un diagnóstico analizando distintas variables que componen al dataset, con la finalidad de obtener insights relevantes para posteriormente crear un modelo que nos permita conocer si la resolución por parte de las compañías ante una problemática o queja por parte de sus usuarios será satisfactoria para ellos, o si habrá descontento y buscarán luchar por una resolución distinta.<p/>

In [2]:
#Asignar la ruta del archivo y descargarlo utilizando el modulo "gdwon",ya que se trata de un dataset bastante pesado.
url = 'https://drive.google.com/file/d/11dX_mR6xWKubBKsgQyhDdN2R6S8oAV4G/view?usp=sharing'
path_origen ='https://drive.google.com/uc?id=' + url.split('/')[-2]

#Generar carpeta data si no existe, y descargar data cruda del origen
if not os.path.exists(RAW_DATA_DIR):
    os.makedirs(RAW_DATA_DIR)
    print(f'Se creo el directorio: {RAW_DATA_DIR}')
  
# Importar el conjunto de datos  
try:
    raw_data_csv = gdown.download(path_origen, RAW_DATA_PATH, quiet=False)
    print(f'Se descargó el conjunto de datos de manera existosa en la ruta: {RAW_DATA_PATH}')
except:
    raise ImportError("ERROR al importar los datos.")

Downloading...
From (original): https://drive.google.com/uc?id=11dX_mR6xWKubBKsgQyhDdN2R6S8oAV4G
From (redirected): https://drive.google.com/uc?id=11dX_mR6xWKubBKsgQyhDdN2R6S8oAV4G&confirm=t&uuid=17918a53-83cf-4a42-a055-07976918421c
To: c:\Users\dancm\OneDrive\Documentos\Courses\Carrera Data Scientist\Data Science\Desafíos\17. Proyecto Final\data\raw\raw_data.csv
100%|██████████| 175M/175M [00:18<00:00, 9.31MB/s] 

Se descargó el conjunto de datos de manera existosa en la ruta: c:\Users\dancm\OneDrive\Documentos\Courses\Carrera Data Scientist\Data Science\Desafíos\17. Proyecto Final\data\raw\raw_data.csv


## 2.2 Exploración Inicial

<p style="text-align: justify;">A continuación se creará el DataFrame principal, el cual contiene todos los datos "en crudo" del dataset, los cuales se traducen en la totalidad de las quejas presentadas a la CPFB durante un periodo de tiempo determinado con diversas características que serán la base para el modelo que se busca desarrollar. Cabe mencionar que después de una exploración bastante profunda, el DataFrame original se someterá a un proceso de limpieza y de ingeniería de atributos para buscar explotar al máximo la información que pueda brindar para el modelo, por lo que tanto la estructura del DataFrame como las mismas variables que se observan ahora pueden ir cambiando a lo largo del desarrollo del modelo.<p/>

In [3]:
#Leer el archivo y crear el DataFrame.
df = pd.read_csv(raw_data_csv, index_col = 17, low_memory=False)
df.head(5)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?
complaint_id,,,,,,,,,,,,,,,,,
511074,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes
511080,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes
510473,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No
510326,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes
511067,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes


In [4]:
# Observar estructura del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 555957 entries, 511074 to 984116
Data columns (total 17 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   date_received                 555957 non-null  object
 1   product                       555957 non-null  object
 2   sub_product                   397635 non-null  object
 3   issue                         555957 non-null  object
 4   sub_issue                     212622 non-null  object
 5   consumer_complaint_narrative  66806 non-null   object
 6   company_public_response       85124 non-null   object
 7   company                       555957 non-null  object
 8   state                         551070 non-null  object
 9   zipcode                       551452 non-null  object
 10  tags                          77959 non-null   object
 11  consumer_consent_provided     123458 non-null  object
 12  submitted_via                 555957 non-null  object
 13 

<p style="text-align: justify;">Se puede observar un DataFrame con una cantidad relevante de variables, las cuales podrían ser beneficiosas o no para el modelo que se plantea llevar a cabo, por lo que es de gran importancia hacer un análisis y profundizar en cada una de ellas. A continuación, se presenta una breve descripción sobre cada una de las columnas que componen el dataset:<p/>

|      Variable (Col.)      |      Tipo      |      Descripción      |
|:------------------:|:------------------:|:------------------:|
|   *complaint_id*     |     **Nominal**          |     Identificador único para cada una de las quejas o registros (índice).          |
|  *date_received*     |     **Intérvalo**          |     Fecha en la cuál la Oficina para la Protección Financiera del Consumidor recibió la queja.          |
|     *product*          |      **Nominal**          |     Categoría que hace referencia al producto financiero sobre el cuál se emitió la queja.          |
|     *sub_product*          |      **Nominal**          |     Subcategoría que detalla más a fondo el producto o servicio que produjo la inconformidad.          |
|     *issue*          |      **Nominal**          |     Indica de manera categórica el problema que tuvo el usuario.          |
|     *sub_issue*          |      **Nominal**          |     Indica de manera más profunda el problema que tuvo el usuario.          |
|     *consumer_complaint_narrative*          |      **Nominal**          |     Narrativa del cliente sobre el problema que lo llevó a realizar la queja.          |
|     *company_public_response*          |      **Nominal**          |     Nos muestra de manera categorizada la respuesta pública opcional de la compañía sobre la queja.          |
|     *company*          |      **Nominal**          |     Compañía que presta el servicio financiero al usuario sobre el cual se realizó la queja.          |
|     *State*          |      **Nominal**          |     Estado en el que se presentó la queja (incluyendo colonias americanas fuera del país).          |
|     *zipcode*          |      **Nominal**          |     Código postal del cliente que presentó la queja.          |
|     *tags*          |      **Nominal**          |     Indica si se trata de una persona de la tercera edad, de una persona que sirve a las fuerzas armadas o ambas.          |
|     *consumer_consent_provided*          |      **Nominal**          |     Muestra de manera categórica el estatus del consentimiento del usuario, es decir, si optó por publicar su narrativa de queja.          |
|     *submitted_via*          |      **Nominal**          |     El medio a través del cual el cliente presentó la queja.          |
|     *date_sent_to_company*          |      **Intérvalo**          |     Fecha en la que la Oficina para la Protección Financiera del Consumidor notificó la queja a la compañía.          |
|     *company_response_to_consumer*          |      **Nominal**          |     Muestra de manera categorizada la manera como respondió la compañía al usuario sobre la queja.          |
|     *timely_response*          |      **Nominal**          |     Menciona si la compañía dio una respuesta oportuna.          |
|     *consumer_disputed?*          |      **Nominal**          |     Indica si el consumidor ha disputado la respuesta proporcionada por la compañía. Si el valor es "Sí", significa que el consumidor no estuvo de acuerdo con la respuesta de la compañía. Si el valor es "No", significa que el consumidor aceptó la respuesta de la compañía.          |

Se han obtenido algúnas primeras impresiones luego de haber revisado detalladamente cada una de las columnas del dataset:


- <p style="text-align: justify;">La columna <b><i>subproduct</i></b> cuenta con una gran cantidad de valores nulos, por lo que se podría evaluar basar el análisis únicamente con el producto principal. Lo mismo sucede con la columna "subissue", por lo que se podría basar en el problema principal.<p/>


- <p style="text-align: justify;">La columna <b><i>consumer_complaint_narrative</i></b> presenta la narrativa del usuario al presentar la queja, sin embargo, cuenta con una gran cantidad de valores nulos, se podría prescindir de ella una vez que se hayan explorado los registros (sería bastante útil para un proyecto de text analytics).<p/>


- <p style="text-align: justify;">La columna <b><i>company_public response</i></b> muestra la respuesta pública de la compañía una vez que se revisó la queja, por lo que se puede intuir que los valores nulos se generaron debido a que la compañía decidió no exponer dicha información.<p/>


- <p style="text-align: justify;">Dentro de las columnas <b><i>state</i></b> y <b><i>zipcode</i></b> se pueden encontrar registros vacios. En el caso de los zipcodes se pueden observar de igual manera algunos datos sucios, como por ejemplo valores que termian con caracteres invalidos (zipcode = 55XXX). Por otro lado, en cuanto a los registros con información nula sobre el estado, se verificará si se cuenta con el registro del zipcode, para poder llenarlos de manera idónea. En caso de que no se cuente con esa información, se podría optar por eliminar esos registros, en caso de que se encuentre alta relevancia en estas dos variables.<p/>


- <p style="text-align: justify;">En la columna <b><i>tags</i></b>, se puede entender que todos los valores nulos hacen referencia a que los usuarios no pertenecen a ninguno de los grupos previamente mencionados, por lo que se generará una etiqueta para hacer mención que no pertenecen a estos grupos.<p/>


- <p style="text-align: justify;">Finalmente, en la columna <b><i>consumer_consent_provided</i></b> sucede algo similar a la respuesta pública de la empresa, y en caso de no ser relevante se eliminará y no será tomada en cuenta en el modelo.<p/>